In [ ]:
using Distributions, Plots, Random, LinearAlgebra, StatsBase, LaTeXStrings

In [ ]:
function sample_robot_kinematics(x, u, Δt, ϵ)
    # Extract the pose and velocity from the state and control vector
    X, Y, θ, v, ω = x[1], x[2], x[3], u[1], u[2]
    
    # Compute the variance in the velocities
    σ²_v = ϵ[1] * v^2 + ϵ[2] * ω^2
    σ²_ω = ϵ[3] * v^2 + ϵ[4] * ω^2
    
    # Compute the variance in the final perturbation to the orientation
    σ²_γ = ϵ[5] * v^2 + ϵ[6] * ω^2
    
    # Compute the velocities with the sampled errors
    v̂ = v + rand(Normal(0., σ²_v))
    ω̂ = ω + rand(Normal(0., σ²_ω))
    
    # Compute the perturbation to the final orientation
    γ̂ = rand(Normal(0., σ²_γ))
    
    # Compute the final pose
    X̂ = X - (v̂ / ω̂) * sin(θ) + (v̂ / ω̂) * sin(θ + ω̂ * Δt)
    Ŷ = Y + (v̂ / ω̂) * cos(θ) - (v̂ / ω̂) * cos(θ + ω̂ * Δt)
    θ̂ = θ + ω̂ * Δt + γ̂ * Δt
        
    # Return the final pose
    return [X̂, Ŷ, θ̂]
end

In [ ]:
function plot_robot(x, r, color)
    # Angles to plot the robot as a circle
    ϕs = 0:0.001:2π
    
    # Plot the robot shape
    plot!(x[1] .+ r*sin.(ϕs), x[2] .+ r*cos.(ϕs), seriestype=[:shape,], label="", alpha=0.25, color=color)
    
    # Plot a small line on the robot to show its orientation
    plot!([x[1], x[1] + r * cos(x[3])], [x[2], x[2] + r * sin(x[3])], color="black", label="", alpha=0.25)
end

In [ ]:
function plot_map(map)
    # Plot each object in the map
    for obj in map
        # Plot the object
        plot!(obj[:,1], obj[:,2], label="", color="black", seriestype=[:shape,], alpha=0.25)
    end
end

In [ ]:
function create_box_obj(loc, angle)
    # Create a unit box
    box = [[0., 0.] [1., 0.] [1., 1.] [0., 1.] [0., 0.]]
    
    # Construct the rotation matrix for the angle
    R = [[cos(angle), -sin(angle)] [sin(angle), cos(angle)]]'
    
    # Apply the rotation to the unit box
    box = (R * box)'
    
    # Move the box to the final location
    box[:,1] = box[:,1] .+ loc[1]
    box[:,2] = box[:,2] .+ loc[2]
    
    # Return the constructed box
    return box
end

In [ ]:
function ray_segment_intersection(o, d, pt1, pt2)
    # Compute three intermediate vector quantities
    v1 = o - pt1
    v2 = pt2 - pt1
    v3 = [-d[2], d[1]]
    
    # Compute the denominator of the determinative expression
    denom = dot(v2, v3)
    
    # Check if the segment is parallel to the ray
    if abs(denom) < 1e-7
        return Inf
    end
    
    # Compute the intersection point of the ray and the segmen
    t1 = (v2[1]*v1[2] - v2[2]*v1[1]) / denom
    t2 = dot(v1, v3) / denom
    
    # Determine if the ray intersects the segment
    if t1 ≥ 0 && 0 ≤ t2 ≤1
        return t1
    else
        return Inf
    end
end

In [ ]:
function get_distances(x, robot)
    # Get the orientation angle of the robot
    θ = x[3]
    
    # Get the center of the robot
    o = x[1:2]
    
    # Construct the angles of the sensor directions in the robot grame
    dϕ = 2π / robot.num_sensors
    ϕs = collect(0:dϕ:2π-1e-12)
    
    # Allocate a list of actual ranges
    ranges = []
    
    # Compute the range along each sensor direction
    for ϕ in ϕs
        # Compute the sensor direction in the global frame
        ψ = θ + ϕ
        d = [cos(ψ), sin(ψ)]
        
        # Allocate a list for all the intersections with segments in the map
        distances = []
        
        # Iterate over each object in the map
        for obj in robot.map
            # Iterate over each edge in the object
            for i in 1:size(obj, 1) - 1
                # Compute the intersection of the sensor ray with the edge
                push!(distances, ray_segment_intersection(o, d, obj[i,:], obj[i+1,:]))
            end
        end
        
        # Add the closest intersection distance to the list of ranges for the sensors
        push!(ranges, minimum(distances))
    end
    
    # Return the ranges along each sensor direction
    return ranges
end

In [ ]:
function p_hit(zk, zk_star, zk_max, σ_hit)
    # Normal distribution modelling correct range with local noise
    N = Normal(zk_star, σ_hit^2)
    
    # Compute the normalization constant
    η = 1 / (cdf(N, zk_max) - cdf(N, 0))
    
    # Check if the actual range is beyond the sensor range
    if zk_star ≥ zk_max
        # Return a max sensor reading with probability 1
        return zk == zk_max ? 1 : 0
    end
    
    # Return the probability
    return 0 ≤ zk ≤ zk_max ? η * pdf(N, zk) : 0
end

function p_short(zk, zk_star, λ_short)
    # Exponential distribution to model a short measurement
    E = Exponential(1 / λ_short)
    
    # Compute the normalization constant
    η = 1 / (cdf(E, zk_star) - cdf(E, 0))
    
    # Return the probability
    return 0 ≤ zk ≤ zk_star ? η * pdf(E, zk) : 0
end

function p_max(zk, zk_max)
    # Return the probability when a sensor failure happens
    return zk == zk_max ? 1 : 0
end

function p_rand(zk, zk_max)
    # Return the probability when an unexplainable measurement is made
    return 0 ≤ zk ≤ zk_max ? 1 / zk_max : 0
end

struct sensor_model
    zk_max
    σ_hit
    λ_short
    weights
end

function p_sensor(zk, zk_star, model)
    # Compute the probability contribution from each type of measurement/failure mode
    ps = [p_hit(zk, zk_star, model.zk_max, model.σ_hit), 
          p_short(zk, zk_star, model.λ_short),
          p_max(zk, model.zk_max),
          p_rand(zk, model.zk_max)]
    
    # Return the weighted sum of the probabilities
    return model.weights' * ps
end

function p_sensor_array(z, x, robot)
    # Compute what the actual ranges would be for state "x" using the map
    z_star_bel = get_distances(x, robot)
    
    # Initialize the the probability as 1
    p = 1
    
    # Update the probability with each sensor measurement
    for k = 1:robot.num_sensors
        p *= p_sensor(z[k], z_star_bel[k], robot.sensor)
    end
    
    # Return the probability
    return p
end

function sample_sensor(zk_star, model; dzk=0.01)
    # Compute the CDF of the sensor probability distribution
    sensor_cdf = cumsum([p_sensor(zk, zk_star, model) * dzk for zk in 0:dzk:model.zk_max+dzk])
     
    # Perform the inverse CDF trick through a search
    idx = argmin(abs.(sensor_cdf .- rand()))

    # Return the sampled sensor measurement
    return dzk * (idx - 1)
end

In [ ]:
function particle_filter(xs, u, z, robot, Δt, ϵ; use_update_step=true)
    # Transform each particle using the kinematics model
    xs = [robot.sample_kinematics(x, u, Δt, ϵ) for x in xs]
        
    # Perform the update step using the sensor measurements
    if use_update_step
        # Compute the particle importance factor
        w = Weights([p_sensor_array(z, x, robot) for x in xs])
        
        # Perform the importance sampling step
        xs = xs[sample(collect(1:length(xs)), w, length(xs))]
    end
    
    # Return the new particles
    return xs
end

In [ ]:
struct robot_model
    map
    sample_kinematics
    u_func
    sensor
    num_sensors
    r
end

In [ ]:
function simulate(x0, robot, Δt, ϵ, num_iter, num_particles; use_update_step=true)
    # Initialize a plot for the results
    plot(aspect_ratio=:equal)
    
    # Initialize the ground truth state
    x_gt = x0
    
    # Intitialize the dead reckoning state
    x_dr = x0
    
    # Initialize the particles at the initial state
    xs = [x0 for _ in 1:num_particles]
    
    # Plot the initial particle distribution
    scatter!([Tuple(x[1:2]) for x in xs[1:num_particles]], markersize=1, markerstrokewidth=0, label="")
    
    # Plot the map
    plot_map(robot.map)
    
    # Plot the initial robot state
    plot_robot(x_gt, robot.r, "blue")
    
    # Perform the simulation
    for iter in 1:num_iter
        # Get the current control from the control function
        u = robot.u_func(Δt * iter)
        
        # Update the ground truth using the kinematics
        x_gt_new = robot.sample_kinematics(x_gt, u, Δt, ϵ)
        
        # Update the dead reckoning using the kinematics with no error
        x_dr_new = robot.sample_kinematics(x_dr, u, Δt, zeros(length(ϵ)))
        
        # Get the actual distances to objects along each sensor direction
        z_star = get_distances(x_gt_new, robot)
        
        # Sample the actual sensor measurements
        z = [sample_sensor(zk_star, robot.sensor) for zk_star in z_star]
        
        # Update the particles using the particle filter
        xs_new = particle_filter(xs, u, z, robot, Δt, ϵ; use_update_step)
        
        # Plot the particles
        scatter!([Tuple(x[1:2]) for x in xs_new[1:num_particles]], markersize=1, markerstrokewidth=0, 
                 label=iter == 1 ? "Belief" : "", color=:green)
        
        # Plot the ground truth robot state
        plot_robot(x_gt_new, robot.r, "blue")
        
        # Plot the trajectory of the ground truth state from the previous iteration
        plot!([x_gt[1], x_gt_new[1]], [x_gt[2], x_gt_new[2]], color="blue", label=iter == 1 ? "Ground Truth" : "")
        
        # Plot the trajectory of the dead reckoning state from the previous iteration
        plot!([x_dr[1], x_dr_new[1]], [x_dr[2], x_dr_new[2]], color="red", label=iter == 1 ? "Dead Reckoning" : "")
        
        # Update the states of the simulation
        xs, x_gt, x_dr = xs_new, x_gt_new, x_dr_new
    end
    
    # Set plot parameters
    plot!(grid=false, axis=nothing, border=:none, foreground_color_legend = nothing)
end

In [ ]:
# Initial state of the robot
x0 = [0, 0, π/3]

# First control sequence, left turn
u1 = [1, 0.5]

# Second control sequence, right turn
u2 = [1, -0.5]

# Robot kinematic model disturbance parameters
ϵ = [0.01, 0.01, 0.01, 0.01, 0.1, 0.1]

# Robot radius
r = 0.175

# Simulation time-step
Δt = 1

# Number of sensors in the sensor array
num_sensors = 16

# Create environment and map
box1 = create_box_obj([-4, 0], π/6)
map = [box1, box2, box3]

# Maximum sensor range
zk_max = 5

# Variance in sensor hit
σ_hit = 0.5

# Rate of early return for sensor
λ_short = 1

# Sensor noise mode weights
weights = [0.7, 0.1, 0.1, 0.1]

# Initialize the sensor
sensor = sensor_model(zk_max, σ_hit, λ_short, weights)

# Initialize the robot
robot = robot_model(map, sample_robot_kinematics, t -> t ≤ 5 ? u1 : u2, sensor, num_sensors, r)

# Set the number of particles to use
num_particles = 1000

In [ ]:
Random.seed!(0)
sensor = sensor_model(5, 0.5, 1, [0.7, 0.1, 0.1, 0.1])
zk_star = 3
histogram([sample_sensor(zk_star, sensor) for _ in 1:10_000], normed=true, bins=50, label="",
          xticks=([0, zk_star, 5], [L"0", L"z_{k}^{*}", L"z_{k}^{\max}"]),
          xtickfont=14, xlims=(-0.5, 5.5), ylim=(0, 1.2), yaxis=false, grid=false,
          lcolor=:blue, dpi=500)
plot!(0:0.01:sensor.zk_max, zk -> p_sensor(zk, zk_star, sensor), label="", linewidth=2)
savefig("sensor-model.png")

In [ ]:
# Simulate the robot without the sensor updates
Random.seed!(1)
simulate(x0, robot, Δt, ϵ, 5, num_particles; use_update_step=false)
plot!(xlims=(-6, 1), ylims=(-1, 4), legend=:topleft, dpi=500)
savefig("robot-no-sensor.png")

In [ ]:
# Simulate the robot with the sensor updates
Random.seed!(1)
simulate(x0, robot, Δt, ϵ, 5, num_particles; use_update_step=true)
plot!(xlims=(-6, 1), ylims=(-1, 4), legend=:topleft, dpi=500)
savefig("robot-with-sensor.png")

In [ ]:
# Add more obstacles to the environment
box2 = create_box_obj([-1, 5], π/3)
box3 = create_box_obj([-7, 6], π/5)
map = [box1, box2, box3]
robot = robot_model(map, sample_robot_kinematics, t -> t ≤ 5 ? u1 : u2, sensor, num_sensors, r)

# Simulate the robot in a larger environment with complex move
Random.seed!(1)
simulate(x0, robot, Δt, ϵ, 10, num_particles; use_update_step=true)
plot!(xlims=(-10, 1), ylims=(-1, 8), legend=:left, dpi=500)
savefig("robot-long.png")